In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK047095,Karma Resorts,Other,Past Guests,Karma Kandara,PF257291,Ms,Alexa,Eickhoff,Female,...,2024-11-29 00:00:00,5,TO,TO,RETAIL,2,0,0,Email and Phone,NO
1,KK064414,Karma Resorts,Other,Past Guests,Karma Kandara,PF293489,Mr,Jock,Campbell,Male,...,2024-11-26 00:00:00,3,HOUSE_USE,VIP,COMP,2,0,0,Email and Phone,NO
2,KB046029,Karma Resorts,Other,Past Guests,Karma Bavaria,PF277731,Mr,Chandler,Cunningham-South,Male,...,2024-11-28 00:00:00,2,COMP,GSTAFF,COMP,2,0,0,Email and Phone,NO
3,KK064343,Karma Resorts,Other,Past Guests,Karma Kandara,PF244666,Ms,Isabella,Duric,Female,...,2024-11-24 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,10,0,0,Email and Phone,NO
4,KB045467,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291607,Mrs,Leisa,Williams,Female,...,2024-11-27 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,KB043717,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,Firma Humbaur,Gmbh,Unknown,...,2024-11-27 21:30:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
72,KK048766,Karma Resorts,Other,Past Guests,Karma Kandara,<NA>,<NA>,Feifang,Wedding,Unknown,...,2024-12-01 00:00:00,2,WEDDING,WEDDING,GROUP,8,0,0,Email Only,NO
73,KB043716,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,Firma Humbaur,Gmbh,Unknown,...,2024-11-29 18:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
74,KB042821,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,<NA>,Sandra Saam,Unknown,...,2024-12-01 00:00:00,1,OTA,OTA,RETAIL,1,2,0,Email Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
13,KB042111,Karma Resorts,Other,Past Guests,Karma Bavaria,PF272392,Frau,Stefanie,Hanka,Female,...,2024-11-26 18:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
18,KB045378,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-11-24 19:00:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
20,KB045379,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-11-24 17:00:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
21,KB040425,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-12-01 19:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
22,KB040424,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-11-24 19:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
23,KSH021331,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF294086,Mr,Chris,Ward,Male,...,2024-11-29 00:00:00,2,COMP,STAFF,COMP,1,0,0,Email Only,NO
26,KK061246,Karma Resorts,Other,Past Guests,Karma Kandara,PF286776,Ms,Dayu,Kariani,Female,...,2024-11-24 00:00:00,1,COMP,STAFF,COMP,6,0,0,Email Only,NO
48,KSH021479,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF289623,Mr,Adrian,Wilmers,Male,...,2024-11-29 00:00:00,1,COMP,VIP,COMP,1,0,0,Email Only,NO
51,KSH021259,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293841,Mr,James,Barnes,Male,...,2024-11-29 00:00:00,1,DIR,CB,GROUP,1,0,0,Email Only,NO
55,KSH021255,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293837,Mr,Andrew,Robson,Male,...,2024-11-29 00:00:00,1,DIR,CB,GROUP,1,0,0,Email Only,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK047095,Karma Resorts,Other,Past Guests,Karma Kandara,PF257291,Ms,Alexa,Eickhoff,Female,...,2024-11-29 00:00:00,5,TO,TO,RETAIL,2,0,0,Email and Phone,NO
1,KK064414,Karma Resorts,Other,Past Guests,Karma Kandara,PF293489,Mr,Jock,Campbell,Male,...,2024-11-26 00:00:00,3,HOUSE_USE,VIP,COMP,2,0,0,Email and Phone,NO
2,KB046029,Karma Resorts,Other,Past Guests,Karma Bavaria,PF277731,Mr,Chandler,Cunningham-South,Male,...,2024-11-28 00:00:00,2,COMP,GSTAFF,COMP,2,0,0,Email and Phone,NO
3,KK064343,Karma Resorts,Other,Past Guests,Karma Kandara,PF244666,Ms,Isabella,Duric,Female,...,2024-11-24 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,10,0,0,Email and Phone,NO
4,KB045467,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291607,Mrs,Leisa,Williams,Female,...,2024-11-27 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO
5,KK064196,Karma Resorts,Other,Past Guests,Karma Kandara,PF293210,Mr,Eassam,Mohammed,Male,...,2024-12-01 00:00:00,5,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
6,KB044203,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284640,empty,Casper,Sorensen,Unknown,...,2024-12-01 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
7,KB045341,Karma Resorts,Other,Past Guests,Karma Bavaria,PF290962,Frau,Katrin,Goller-Englberger,Female,...,2024-11-24 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
8,KB045836,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293803,Herr,Daniel,Dienlin,Unknown,...,2024-11-24 00:00:00,2,DIR,TEL,RETAIL,2,0,1,Email and Phone,NO
9,KB046055,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294627,empty,Matthias,Haselbock,Unknown,...,2024-12-01 00:00:00,2,OTA,OTA,RETAIL,2,3,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
19,KK059561,Karma Resorts,Other,Past Guests,Karma Kandara,PF283192,Mr,Ramchandran,Menon,Male,...,2024-12-01 00:00:00,7,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
36,KK060781,Karma Resorts,Other,Past Guests,Karma Kandara,PF077919,Mr,Bryan John,Dyer,Male,...,2024-12-01 00:00:00,6,HOUSE_USE,V_OWNER,COMP,8,0,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KLM007421,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF294148,Mr,Dan,Buys,Unknown,...,2024-11-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KLM007432,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF294340,Miss,Gemma,Watson,Unknown,...,2024-11-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KLM005118,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF245792,Mrs,Jacqui,Flavell,Unknown,...,2024-11-24 00:00:00,3,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
3,KB045529,Karma Resorts,Other,Past Guests,Karma Bavaria,PF292164,Mr,Jeffrey,Blank,Male,...,2024-11-24 00:00:00,8,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
4,KLM004135,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF229146,empty,Angela,Simpson,Unknown,...,2024-11-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
5,KSH021217,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293733,Mr,Gary,Osben,Unknown,...,2024-11-24 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
6,KSH017500,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF269181,empty,Willis-Hunt,Natasha,Unknown,...,2024-11-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KLM007435,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF294343,Mrs,Kateryna,Chernyshova,Unknown,...,2024-11-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB045823,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293735,Frau,Zahnow,Michaela,Female,...,2024-11-24 00:00:00,2,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
9,KB045948,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294344,Ms,Dariia,Podshivalova,Unknown,...,2024-11-24 00:00:00,1,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar\AppData\Local\Temp\ipykernel_18676\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KLM007421,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF294148,Mr,Dan,Buys,Unknown,...,2024-11-24,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KB045341,Karma Resorts,Other,Past Guests,Karma Bavaria,PF290962,Frau,Katrin,Goller-Englberger,Female,...,2024-11-24,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
2,KK064369,Karma Resorts,Other,Past Guests,Karma Kandara,PF293435,Mr,Brett Timothy,Maxwell,Male,...,2024-11-24,7,HOUSE_USE,V_OWNER_NRB,COMP,4,2,0,Email and Phone,NO
3,KB045836,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293803,Herr,Daniel,Dienlin,Unknown,...,2024-11-24,2,DIR,TEL,RETAIL,2,0,1,Email and Phone,NO
4,KB041851,Karma Resorts,Other,Past Guests,Karma Bavaria,PF270692,Herr,Dominik,Dienlin,Male,...,2024-11-24,2,OTA,OTA,RETAIL,2,0,1,Email and Phone,NO
5,KB045948,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294344,Ms,Dariia,Podshivalova,Unknown,...,2024-11-24,1,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
6,KB045823,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293735,Frau,Zahnow,Michaela,Female,...,2024-11-24,2,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
7,KK064343,Karma Resorts,Other,Past Guests,Karma Kandara,PF244666,Ms,Isabella,Duric,Female,...,2024-11-24,2,HOUSE_USE,V_OWNER_NRB,COMP,10,0,0,Email and Phone,NO
8,KSH017500,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF269181,,Willis-Hunt,Natasha,Unknown,...,2024-11-24,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
9,KSH021217,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293733,Mr,Gary,Osben,Unknown,...,2024-11-24,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 76
Jumlah Data Yang Duplicate : 19
Jumlah Data Setelah Hapus Duplicate : 57
Jumlah Data Yang Termasuk Member: 2
Jumlah Data Setelah Hapus Member : 55
Jumlah Data Akhir : 55


In [19]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)